In [ ]:
import os
import numpy as np
import scipy.io as sio
import pickle

In [ ]:
# np.random.seed(42)  # Set random seed

# from scipy.stats import zscore
# def pearson_corr(X,Y):
#     return np.mean(zscore(X)*zscore(Y),0)

# def compute_pearson_from_preds_numpy(pred_path, expt_setting):
#     print('compute_pearson_from_preds_numpy:', pred_path, expt_setting)
#     loaded = np.load(pred_path, allow_pickle=True)
#     preds_t_per_feat = loaded.item()['preds_t']                             # (1191, ~27905)
#     test_t_per_feat = loaded.item()['test_t']
    
#     if expt_setting == 'full':  # Use the full set
#         pass
#     else:
#         TR_one_hot_pkl_path = f'./data/TR_one_hot_for_features/{expt_setting}.pkl'
#         TR_one_hot = pickle.load(open(TR_one_hot_pkl_path, 'rb'))
#         TR_indices = np.where(TR_one_hot==1)[0]                             # List of TR indices: [10, 13,...]
#         TR_indices = np.random.choice(TR_indices, size=162, replace=False)  # Randomly select 162 TR indices, with set seed
#         TR_indices = np.sort(TR_indices)
        
#         preds_t_per_feat = preds_t_per_feat[TR_indices, :]                  # Extract from only the desired TRs
#         test_t_per_feat =  test_t_per_feat[TR_indices, :]
    
#     pearson_per_voxel = pearson_corr(preds_t_per_feat, test_t_per_feat)     # (~27905,)
#     pearson_scalar = np.mean(pearson_per_voxel)                             # 1 scalar number
#     return pearson_scalar, pearson_per_voxel

# pred_path = '2-encoding_predictions/led-base/predict_F_with_led-base_layer_6_len_100.npy'
# expt_setting = 'Characters'
# print(compute_pearson_from_preds_numpy(pred_path, expt_setting))

In [ ]:
# pred_path = '2-encoding_predictions/led-booksum/predict_H_with_led-booksum_layer_1_len_20.npy'
# loaded = np.load(pred_path, allow_pickle=True)
# preds_t_per_feat = loaded.item()['preds_t']
# test_t_per_feat = loaded.item()['test_t']
# print('preds:', preds_t_per_feat.shape)

In [ ]:
from utils.utils import CV_ind

test_word_indices_1 = [ 2, 20, 21, 39, 101, 132, 139, 150, 158, 188, 443, 451, 464, 569, 570, 601, 626, 633, 645, 648, 955, 956, 1222, 1244, 1272, 1298, 1306, 1316, 1319, 1321, 1343, 1347, 1352, 1356, 1369, 1500, 1504, 1515, 1523, 1552, 1556, 1591, 1593, 1604, 1612, 1615, 1796, 1811, 2667, 2690, 2696, 2702, 2704, 2708, 2746, 2752, 2755, 2777, 2792, 2794, 2805, 2822, 2895, 2919, 2942, 2987, 3093, 3144, 3162, 3831, 3883, 3895, 3898, 3930, 4218, 4230, 4249]

def convert_word_indices_to_TR_one_hot(word_indices, SKIP_WORDS=20, END_WORDS=5176):   # list word indices ==> [20, 21, 39, ...]
    time = np.load('./data/fMRI/time_fmri.npy')                 # (1351,) => [0, 2, 4, ...]
    runs = np.load('./data/fMRI/runs_fmri.npy')                 # (1351,) => [1,...,2,...,3,...,4]
    time_words = np.load('./data/fMRI/time_words_fmri.npy')     # (5176,) => [20., 20.5, ..., 2693.0]
    time_words = time_words[SKIP_WORDS:END_WORDS]               # (5156,) => [30., 30.5, ..., 2693.0]

    # Find what TR each word belongs to
    words_id = np.zeros([len(time_words)],dtype=int)            # (5156,) => [0, 0, ..., 0]
    for i in range(len(time_words)):                
        words_id[i] = np.where(time_words[i]> time)[0][-1]      # (5156,) => [14, ..., 1346] first word belongs to TR 14

    word_one_hot = np.zeros((5176))
    word_one_hot[word_indices] = 1                              # (5176,) ==> [1, 0, 0, 1, 1, ...]
    word_one_hot = word_one_hot[SKIP_WORDS:END_WORDS]           # (5156,) ==> [0, 1, 0, 0, 1, ...]
    
    TR_indices = words_id[np.where(word_one_hot == 1)]          # [14, 15, 19, ..., 35]
    TR_one_hot = np.zeros((time.shape[0]))
    TR_one_hot[TR_indices] = 1                                  # (1351,) ==> [0, 1, 0, 0, 1, ...]

    # remove the edges of each run
    TR_one_hot_runs = [TR_one_hot[runs==i][20:-15] for i in range(1,5)]     # (1211,) ==> IMPORTANT how 1351 --> 1211
    TR_one_hot = np.array([])
    for i in range(len(TR_one_hot_runs)):
        TR_one_hot = np.concatenate([ TR_one_hot, TR_one_hot_runs[i] ])
    
    final_TR_one_hot = np.array([])
    n_folds = 4
    skip = 5
    ind = CV_ind(len(TR_one_hot), n_folds=n_folds)              # (1211,) => [0., 0., 0., ..., 3., 3., 3.]
    for ind_num in range(n_folds):
        test_ind = ind==ind_num                                 # (1211,) => [ True,  True,  True, ..., False, False, False]
        TR_one_hot_fold = np.array(TR_one_hot)[test_ind]
        if ind_num == 0 or ind_num == n_folds-1:
            pass
        else:
            TR_one_hot_fold = TR_one_hot_fold[skip:-skip]

        final_TR_one_hot = np.concatenate([ final_TR_one_hot, TR_one_hot_fold ])
    print('final_TR_one_hot shape:', final_TR_one_hot.shape[0])
    print('final_TR_one_hot number of 1s:', len(np.where(final_TR_one_hot == 1)[0]))
    return final_TR_one_hot
    
convert_word_indices_to_TR_one_hot(test_word_indices_1)

In [ ]:
HP_text_array = np.load(os.getcwd() + '/data/stimuli_words.npy')

story_features_mat = sio.loadmat('./data/story_features.mat')
story_features = story_features_mat['features'][0]

n_upper_features = len(story_features)
for ind_upper_feature in range(0, n_upper_features):
    upper_features = story_features[ind_upper_feature]              # Data for Speech features
    upper_feature_name = upper_features[0][0]
    print(f'Index {ind_upper_feature}, Upper feature name:', upper_feature_name)

In [ ]:
def skip_sub_feature(upper_feature_name, sub_feature_name):
    if upper_feature_name == 'Motion':  # Strange data labeling for Motion
        if ('sticky' in sub_feature_name) and (not sub_feature_name == 'collidePhys_sticky'):
            return False
        else:
            return True
    
    # Commented out because I decided to go with the original authors' annotations
    # if upper_feature_name == 'Emotion':   
    #     if sub_feature_name in ['commanding', 'dislike', '']:
    #         return True
        
    skip_bool = True
    skip_bool = ('sticky' in sub_feature_name)
    
    return skip_bool

def keep_names(word_indices, HP_text_array):
    names  = ['Draco', 'Malfoy', 'Filch', 'Harry', 'Potter', 'Hermione', 'Granger', 'Hooch', 'McGonagall']
    names += ['Neville','Longbottom', 'Peeves', 'Ron', 'Wood', ]
    names = [name.lower() for name in names]
    
    ret_indices = []
    for ind in word_indices:
        word = HP_text_array[ind].lower()   # Just match lowercase
        for name in names:
            if name in word:                # Used "in" so words like Harry'd've would still be included
                ret_indices.append(ind)
                break
    
    return ret_indices

In [ ]:
import sys

n_upper_features = len(story_features)

for ind_upper_feature in range(1, n_upper_features):

    upper_features = story_features[ind_upper_feature]              # Data for Characters features
    upper_feature_name = upper_features[0][0]                       # Characters
    sub_feature_name_list = [ f[0] for f in upper_features[1][0] ]  # ['draco', 'flich', 'harry', ...]
    n_sub_features = len([ f[0] for f in upper_features[1][0] ])    # 10

    out_txt_path = f'0-logs/HP_features/{upper_feature_name}.txt'
    out_file = open(out_txt_path, 'w')
    sys.stdout = out_file
    
    print('Upper feature name:', upper_feature_name)
    print('Sub-features:', sub_feature_name_list)
    print('n_sub_features:', n_sub_features)
    print('Numeric array:', upper_features[2].shape)
    print('===\n')

    if upper_feature_name == 'Word_Num':
        continue
    
    upper_feature_word_indices = []
    for ind_sub_feature in range(n_sub_features):
        sub_feature_name = sub_feature_name_list[ind_sub_feature]
        print('\nSub-feature name:', sub_feature_name)
        if skip_sub_feature(upper_feature_name, sub_feature_name):
            print("=== SKIP this sub-feature ===")
            # continue
        
        sub_feature_ndarray = upper_features[2][:, ind_sub_feature]
        print('sub_feature_ndarray:', sub_feature_ndarray.shape)
        
        unique_nums = np.unique(sub_feature_ndarray)
        print('unique_nums:', unique_nums)
        
        

        for unique_num in unique_nums:
            print(f'--- unique_num: {unique_num} ---')
            if unique_num == 0: # Number 0 means absence of feature, so ignore
                continue
            if skip_sub_feature(upper_feature_name, sub_feature_name):
                print("=== SKIP this sub-feature ===")
            
            word_indices = np.where( sub_feature_ndarray == unique_num )[0]
            word_indices = word_indices.tolist()
            if upper_feature_name == 'Characters':
                word_indices = keep_names(word_indices, HP_text_array)

            print('# of words:', len(word_indices))
            print('word_indices:', word_indices)
            print(HP_text_array[word_indices])
            
            if skip_sub_feature(upper_feature_name, sub_feature_name):
                continue

            # Add word indices to upper feature list
            upper_feature_word_indices += word_indices
            print('upper_feature_word_indices:', upper_feature_word_indices)

    upper_feature_word_indices = list(set(upper_feature_word_indices))
    print('upper_feature_word_indices:', len(upper_feature_word_indices))
    print('upper_feature_word_indices:', (upper_feature_word_indices))
    print('final words:', HP_text_array[upper_feature_word_indices])
    
    TR_one_hot = convert_word_indices_to_TR_one_hot(upper_feature_word_indices)
    print('TR_one_hot:', TR_one_hot)
    
    TR_one_hot_pkl_path = f'./data/TR_one_hot_for_features/{upper_feature_name}.pkl'
    with open(TR_one_hot_pkl_path, 'wb') as handle:
        pickle.dump(TR_one_hot, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

In [ ]:
# === Create TR one-hot for Non-discourse features ===
import numpy as np
import pickle
# Finally, I also want to create a subset of TRs that are excluded from Character, Motion, Emotion
included_TR_one_hot = np.zeros((1191,))
for expt_setting in ['Characters', 'Emotion', 'Motion']:
    expt_TR_one_hot_pkl_path = f'./data/TR_one_hot_for_features/{expt_setting}.pkl'
    expt_TR_one_hot = pickle.load(open(expt_TR_one_hot_pkl_path, 'rb'))
    included_TR_one_hot = included_TR_one_hot + expt_TR_one_hot
    
    print(f'{expt_setting}: {len(np.where(expt_TR_one_hot==1)[0])}')
    
print(f'TRs with 0 marked:', len(np.where(included_TR_one_hot==0)[0]))
print(f'TRs with 1 marked:', len(np.where(included_TR_one_hot==1)[0]))
print(f'TRs with 2 marked:', len(np.where(included_TR_one_hot==2)[0]))
print(f'TRs with 3 marked:', len(np.where(included_TR_one_hot==3)[0]))

# Get those 
non_discourse_TR_one_hot = (included_TR_one_hot==0).astype(int)
print(f'TRs that are marked as non_discourse: {len(np.where(non_discourse_TR_one_hot==1)[0])}')

non_discourse_TR_one_hot_pkl_path = f'./data/TR_one_hot_for_features/Non-Discourse.pkl'
with open(non_discourse_TR_one_hot_pkl_path, 'wb') as handle:
    pickle.dump(non_discourse_TR_one_hot, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
# === Create TR one-hot for Full set ===
import numpy as np
import pickle
full_TR_one_hot = np.ones((1191,))

full_TR_one_hot_pkl_path = f'./data/TR_one_hot_for_features/Full.pkl'
with open(full_TR_one_hot_pkl_path, 'wb') as handle:
    pickle.dump(full_TR_one_hot, handle, protocol=pickle.HIGHEST_PROTOCOL)